# Attack viz

## Set up

In [2]:
import matplotlib
import pandas as pd
import pymysql
%matplotlib inline

In [3]:
connection = pymysql.connect(host='localhost',
                             user='fingerpatch',
                             password='fingerpatch',
                             db='fingerpatch',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

In [4]:
attack_table = pd.read_sql("SELECT * FROM `ubuntu_captures` ",connection)
ground_truth = pd.read_sql("SELECT * FROM `ubuntu_packets` ",connection)

In [11]:
ground_truth.count()[0]

128148

## Cleaning the db remove
Select interesting row and remove duplicate

In [83]:
ground_truth.columns

Index(['capture_id', 'Package', 'Version', 'Architecture', 'Size',
       'Installed-Size', 'Priority', 'Maintainer', 'SHA1', 'Description',
       'parsedFrom', 'Description-md5', 'Bugs', 'Origin', 'MD5sum', 'Depends',
       'Homepage', 'Source', 'SHA256', 'Section', 'Supported', 'Filename',
       'packageMode', '#Depends', 'dist_from_expected_size'],
      dtype='object')

## Mounting an Attack for matching a specific capture to a package.

In [82]:
target = attack_table.loc[0]
target

capture_id                                     1
nb_flows                                       3
Flow1               target->danava.canonical.com
Flow2               danava.canonical.com->target
Flow3               target->danava.canonical.com
Flow4                                       None
Flow5                                       None
nb_Payload_send1                               0
nb_Payload_send2                           67874
nb_Payload_send3                             173
nb_Payload_send4                            None
nb_Payload_send5                            None
Name: 0, dtype: object

In [74]:
EXTRA_SIZE_AVERAGE = 283   # Made from stats about captured packets
EXTRA_SIZE_VARIATION = 5
size_to_match = target['nb_Payload_send2']

In [78]:
def distance_from_expected_average_size(x, size_to_match):
    return abs(size_to_match - x - EXTRA_SIZE_AVERAGE)

In [79]:
ground_truth["dist_from_expected_size"] = ground_truth["Size"].map(lambda x: distance_from_expected_average_size(x, size_to_match))

In [80]:
ground_truth.sort_values(by="dist_from_expected_size")

,capture_id,Package,Version,Architecture,Size,Installed-Size,Priority,Maintainer,SHA1,Description,...,Depends,Homepage,Source,SHA256,Section,Supported,Filename,packageMode,#Depends,dist_from_expected_size
103745,1,libopennebula-java-doc,3.4.1-4.1ubuntu1,all,67592,1194,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,b2449c56f7c0283ec320e83dbbf96164773b39b6,Java bindings for OpenNebula Cloud API (OCA) -...,...,,http://opennebula.org/,opennebula,61b560e07edf80a5fb3a3c59da5aed8e16b460b7c107cf...,universe/doc,,pool/universe/o/opennebula/libopennebula-java-...,source,0,1
39671,1,libopennebula-java-doc,3.4.1-4.1ubuntu1,all,67592,1194,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,b2449c56f7c0283ec320e83dbbf96164773b39b6,Java bindings for OpenNebula Cloud API (OCA) -...,...,,http://opennebula.org/,opennebula,61b560e07edf80a5fb3a3c59da5aed8e16b460b7c107cf...,universe/doc,,pool/universe/o/opennebula/libopennebula-java-...,releases,0,1
41518,1,libshisa-dev,1.0.2-3ubuntu2,amd64,67594,385,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,6cef545cfad241d0c4b1333e9396648a23094c56,Development files for the Shishi Kerberos v5 K...,...,"libshisa0 (= 1.0.2-3ubuntu2), libshishi-dev (=...",http://www.gnu.org/software/shishi/,shishi,7ea40bcc47247bca0449e641d92a9afea23851227bb580...,universe/libdevel,,pool/universe/s/shishi/libshisa-dev_1.0.2-3ubu...,releases,2,3
78227,1,libcloog-isl-dev,0.18.2-1,amd64,67588,377,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,ab29290b9279ffd0eabe34b9a7689fc5a3defd8e,Chunky Loop Generator (development files),...,"libisl-dev (>= 0.11), libgmp-dev, libcloog-isl...",http://www.CLooG.org,cloog,2a291cd9de63a5520da767ceaedbabad10ad88ae2ebc1a...,libdevel,9m,pool/main/c/cloog/libcloog-isl-dev_0.18.2-1_am...,source,4,3
14153,1,libcloog-isl-dev,0.18.2-1,amd64,67588,377,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,ab29290b9279ffd0eabe34b9a7689fc5a3defd8e,Chunky Loop Generator (development files),...,"libisl-dev (>= 0.11), libgmp-dev, libcloog-isl...",http://www.CLooG.org,cloog,2a291cd9de63a5520da767ceaedbabad10ad88ae2ebc1a...,libdevel,9m,pool/main/c/cloog/libcloog-isl-dev_0.18.2-1_am...,releases,4,3
105592,1,libshisa-dev,1.0.2-3ubuntu2,amd64,67594,385,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,6cef545cfad241d0c4b1333e9396648a23094c56,Development files for the Shishi Kerberos v5 K...,...,"libshisa0 (= 1.0.2-3ubuntu2), libshishi-dev (=...",http://www.gnu.org/software/shishi/,shishi,7ea40bcc47247bca0449e641d92a9afea23851227bb580...,universe/libdevel,,pool/universe/s/shishi/libshisa-dev_1.0.2-3ubu...,source,2,3
34921,1,libghc-shakespeare-i18n-prof,1.0.0.2-4build1,amd64,67582,429,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,157bebabbec9d14565590bdc4e6e7ef9a36bdbd5,type-based approach to internationalization; p...,...,libghc-shakespeare-i18n-dev (= 1.0.0.2-4build1...,http://hackage.haskell.org/package/shakespeare...,haskell-shakespeare-i18n,f1968377ff9f4c763d0cc5c1cc5f6d293624e3704cbc59...,universe/haskell,,pool/universe/h/haskell-shakespeare-i18n/libgh...,releases,8,9
98995,1,libghc-shakespeare-i18n-prof,1.0.0.2-4build1,amd64,67582,429,extra,Ubuntu Developers <ubuntu-devel-discuss@lists....,157bebabbec9d14565590bdc4e6e7ef9a36bdbd5,type-based approach to internationalization; p...,...,libghc-shakespeare-i18n-dev (= 1.0.0.2-4build1...,http://hackage.haskell.org/package/shakespeare...,haskell-shakespeare-i18n,f1968377ff9f4c763d0cc5c1cc5f6d293624e3704cbc59...,universe/haskell,,pool/universe/h/haskell-shakespeare-i18n/libgh...,source,8,9
26684,1,gkrellmoon,0.6-5,amd64,67578,320,optional,Ubuntu MOTU Developers <ubuntu-motu@lists.ubun...,49cd0f17854fb19287864d0d199521cd5abbd7e6,Gkrellm Moon Clock Plugin,...,"gkrellm (>= 2.0.0), libatk1.0-0 (>= 1.13.2), l...",,,43e853da209703fc5feab9d14ae18f311bf16efa57b04f...,universe/x11,,pool/universe/g/gkrellmoon/gkrellmoon_0.6-5_am...,releases,18,13
90758,1,gkrellmoon,0.6-5,amd64,67578,320,optional,Ubuntu MOTU Developers <ubuntu-mo

The first one is the one.

In [89]:
ground_truth.sort_values(by="dist_from_expected_size").iloc[0]

capture_id                                                                 1
Package                                               libopennebula-java-doc
Version                                                     3.4.1-4.1ubuntu1
Architecture                                                             all
Size                                                                   67592
Installed-Size                                                          1194
Priority                                                               extra
Maintainer                 Ubuntu Developers <ubuntu-devel-discuss@lists....
SHA1                                b2449c56f7c0283ec320e83dbbf96164773b39b6
Description                Java bindings for OpenNebula Cloud API (OCA) -...
parsedFrom                 packages/archive.ubuntu.com_ubuntu_dists_trust...
Description-md5                             bd5164b179b060e59f8e811d417ecd61
Bugs                              https://bugs.launchpad.net/ubuntu/+filebug

## Dependens discovery

let's take only the packages that have only one depends.

Sort them by ascending size

In [55]:
tmp = ground_truth
tmp["#Depends"] = tmp["Depends"].map(lambda x: 0 if x == "" else len(x.split(",")))
two_dep_first10 = tmp[tmp["#Depends"] == 1].sort_values(by = "Size", ascending=True)[:10]

In [64]:
two_dep_first10.iloc[0]

capture_id                                                         1
Package                                                      readpst
Version                                               0.6.59-1build1
Architecture                                                     all
Size                                                             796
Installed-Size                                                    21
Priority                                                    optional
Maintainer         Ubuntu Developers <ubuntu-devel-discuss@lists....
SHA1                        da3b4b8e1da9e72c3f8a82d03fd620a8590c7186
Description            Converts Outlook PST files to mbox and others
parsedFrom         packages/archive.ubuntu.com_ubuntu_dists_trust...
Description-md5                     3ef2b339ca5bf4fb7de6354b65f7cf59
Bugs                      https://bugs.launchpad.net/ubuntu/+filebug
Origin                                                        Ubuntu
MD5sum                            

Seeking about that depends

In [63]:
ground_truth[ground_truth["Package"] == two_dep_first10.iloc[0]["Depends"]]

,capture_id,Package,Version,Architecture,Size,Installed-Size,Priority,Maintainer,SHA1,Description,...,MD5sum,Depends,Homepage,Source,SHA256,Section,Supported,Filename,packageMode,#Depends
18279,1,pst-utils,0.6.59-1build1,amd64,62092,181,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,7e5afc4156a87f8c02ae369428ffcbb967111bed,tools for reading Microsoft Outlook PST files,...,f75684e6ddb8a1a651a629de9bd42d5d,"libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 ...",http://www.five-ten-sg.com/libpst/,libpst,807a10cc7b8062403018d37a6d19cd210bd9f836ae62d9...,utils,9m,pool/main/libp/libpst/pst-utils_0.6.59-1build1...,releases,7
82353,1,pst-utils,0.6.59-1build1,amd64,62092,181,optional,Ubuntu Developers <ubuntu-devel-discuss@lists....,7e5afc4156a87f8c02ae369428ffcbb967111bed,tools for reading Microsoft Outlook PST files,...,f75684e6ddb8a1a651a629de9bd42d5d,"libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 ...",http://www.five-ten-sg.com/libpst/,libpst,807a10cc7b8062403018d37a6d19cd210bd9f836ae62d9...,utils,9m,pool/main/libp/libpst/pst-utils_0.6.59-1build1...,source,7


Turns out that this Dependens also have its Dependances

In [87]:
sub_dependances = ground_truth[ground_truth["Package"] == two_dep_first10.iloc[0]["Depends"]].iloc[0]["Depends"]
print(sub_dependances)

libc6 (>= 2.14), libgcc1 (>= 1:4.1.1), libgd3 (>= 2.1.0~alpha~), libglib2.0-0 (>= 2.12.0), libgsf-1-114 (>= 1.14.8), libpst4 (>= 0.6.54), libstdc++6 (>= 4.6)


#### Once Downloading the package we can ideed see that the package doesn't just depends on one single package but many subpackages from that dependance:

On the Docker:

`The following extra packages will be installed:
  fontconfig-config fonts-dejavu-core libfontconfig1 libfreetype6 libgd3
  libglib2.0-0 libglib2.0-data libgsf-1-114 libgsf-1-common libjbig0
  libjpeg-turbo8 libjpeg8 libpst4 libtiff5 libvpx1 libx11-6 libx11-data
  libxau6 libxcb1 libxdmcp6 libxml2 libxpm4 pst-utils sgml-base
  shared-mime-info xml-core`
  
`0 upgraded, 27 newly installed, 0 to remove and 32 not upgraded.
Need to get 5664 kB of archives.`


On the attacker:

`historic =  ['target->danava.canonical.com', 'danava.canonical.com->target', 'target->danava.canonical.com']
server_ip =  ['91.189.88.149', '172.100.0.100', '91.189.88.149']
server_name =  ['danava.canonical.com', 'target', 'danava.canonical.com']
received_Payload =  [5671834]
send_Payload =  [0, 4251]`

So if we calculate with the tipical extra_size for each downloaded package that we get on the attacker side and knowing that 5664kB is rounded:


In [96]:
EXTRA_SIZE_AVERAGE * 27 + 5664000

5671641